## Sandbox for marching squares

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from marching_squares import Grid, UnitCell, Lattice
from tables import STATES

In [2]:
def center_ellipse(x,y,r,c):

    draw.ellipse([x - r, y - r, x + r, y + r],fill=c)

def center_rectangle(x,y,l,w,c):
    l = l/2
    w = w/2
    draw.rectangle([x - w, y - l, x + w, y + l],fill=c)

Marching squares

In [7]:
def get_state(a,b,c,d,threshold):

    a = np.floor(a / threshold)
    b = np.floor(b / threshold)
    c = np.floor(c / threshold)
    d = np.floor(d / threshold)

    # probably a better way of doing this
    a = min(a,1)
    b = min(b,1)
    c = min(c,1)
    d = min(d,1)

    s = round(a * 8 + b * 4 + c * 2 + d * 1)
    return s
    
def field_1(x, y, radius):
    # distance from origin
    d = np.sqrt(x ** 2 + y ** 2)
    if d < radius:
        return 1
    else:
        return 0
def field_2(x, y, radius):
    # distance from center point
    d = np.sqrt((x - 1080 / 2) ** 2 + (y - 1080 / 2) ** 2)
    if d < radius:
        return 1
    else:
        return 0

def field_3(x, y, radius, width):
    # distance from center point
    d = np.sqrt((x - width / 2) ** 2 + (y - width / 2) ** 2)
    max_distance = np.sqrt(2 * (width / 2) ** 2)
    v = 1 - d / max_distance

    return max(0, v)

def field_4(x, y, radius, width):
    # distance from center point
    d1 = np.sqrt((x - width / 3) ** 2 + (y - width / 3) ** 2)
    d2 = np.sqrt((x - 2 * width / 3) ** 2 + (y - 2 * width / 3) ** 2)
    max_distance = np.sqrt(2 * (width / 2) ** 2)
    v = 1 - min(d1, d2) / max_distance
    v = v ** 2
    return max(0, v)

In [24]:
def lerp(a, b, t):
    c = a + t * (b - a)
    return c

def find_t(a, b):
    if a + b == 0:
        return 0.5
    t = b / (a + b)
    print(t)
    return t

In [26]:
image_resolution = 1080

img = Image.new('RGB', (image_resolution, image_resolution))
draw = ImageDraw.Draw(img)

grid_divisions = 100
grid_scale = image_resolution / (grid_divisions - 1)


grid = Grid(grid_scale, grid_divisions, grid_divisions)
radius = 200
threshold = 0.5
for point in grid.points:
    v = field_4(point[0], point[1], radius, image_resolution)
    # if v > threshold:
    #     color = "red"
    # else:
    #     color = "gray"
    v = round(v * 255)

    max_distance = np.sqrt(2 * (image_resolution / 2) ** 2)
    scale = grid.scale
    x = point[0]
    y = point[1]

    ## values at points
    v0 = field_4(x, y, radius, image_resolution)
    v1 = field_4(x + scale, y, radius, image_resolution)
    v2 = field_4(x + scale, y + scale, radius, image_resolution)
    v3 = field_4(x, y + scale, radius, image_resolution)

    ## t values
    t0 = find_t(v0, v1)
    t1 = find_t(v1, v2)
    t2 = find_t(v2, v3)
    t3 = find_t(v3, v0)

    ## edge point locations
    a = [x + t0 * scale, y]
    b = [x + scale, y + t1 * scale]
    c = [x + t2 * scale, y + scale]
    d = [x, y + t3 * scale]


    edge_points = [a,b,c,d]

    radius = 400
    state = get_state(v0, v1, v2, v3, threshold)

    edges = STATES[state]

    
    for line in edges:
        p1 = edge_points[line[0]]
        p2 = edge_points[line[1]]
        draw.line([p1[0], p1[1], p2[0], p2[1]], fill="white",width=3)

    # center_ellipse(point[0], point[1], 2, color)
    center_ellipse(point[0], point[1], 4, f'rgb({v},{v},{v})')


img.save('pixels.png')

0.5148090207203098
0.5146027427330007
0.4853972572669994
0.4851909792796903
0.5146027427330007
0.5139715658738297
0.48558227507556
0.485843406874819
0.51441772492444
0.5133705367031343
0.4857478850673124
0.48646383706376717
0.5142521149326875
0.512796304738238
0.4858957300202904
0.4870558293700393
0.5141042699797097
0.5122458853588467
0.4860272766585647
0.48762254404136757
0.5139727233414353
0.5117166071794449
0.4861438436571339
0.4881668001745031
0.5138561563428662
0.5112060685741359
0.48624662518228823
0.4886911236340404
0.5137533748177119
0.5107121016759536
0.4863367107897065
0.489197786679638
0.5136632892102936
0.5102327425316567
0.48641510231887836
0.4896888408029383
0.5135848976811216
0.5097662063674605
0.48648272827933636
0.49016614396088587
0.5135172717206636
0.5093108671317101
0.486540456111556
0.490631383151371
0.513459543888444
0.5088652406421509
0.4865891026159536
0.4910860930924717
0.5134108973840464
0.5084279707891676
0.4866294427719893
0.4915316716249736
0.51337055722801

Lattices

In [6]:
# image_resolution = 1080

# img = Image.new('RGB', (image_resolution, image_resolution))
# draw = ImageDraw.Draw(img)

# def draw_cell(unit_cell, coords):
#     for beam in unit_cell.beams:
#         x1 = coords[beam[0]][0]
#         y1 = coords[beam[0]][1]
#         x2 = coords[beam[1]][0]
#         y2 = coords[beam[1]][1]
#         draw.line([x1, y1, x2, y2], fill="white",width=10)

# unit_cell = UnitCell("SC")
# scale = 100
# unit_cell.x_scale = scale
# unit_cell.y_scale = scale
# lattice = Lattice(unit_cell, 5, 5)
# i = 0
# coords = lattice.return_cell_coords(0,0)
# for coords in lattice.return_all_cell_coords():
#     draw_cell(unit_cell, coords)
# # for coords in lattic coords)
# img.save('pixels.png')